# 1. Imports

In [2]:
from build_model import build_v2g_model

# 2. Model setup

In [3]:
input_variables = ['Q9','Q2','Q8_1','Q8_2']
target_variable = 'Q10_2'  # "I would be interested in installing V2G..."
csv_file = './data/survey_pre_processed_data.csv'

# 3. Model run

In [4]:
model = build_v2g_model(csv_file, input_variables, target_variable)

Accuracy on training data:  0.43837638376383764
Coefficients shape: (5, 4)
Coefficients: [[-2.35285265 -0.09430956 -0.85155887 -0.27600046]
 [-0.79366694 -0.5840581  -0.20669025  0.        ]
 [-1.90083501 -0.0897917   0.03718857 -0.21676575]
 [ 0.67510463 -0.41517289  0.46283426  0.12207956]
 [ 2.85253158  0.79213982  0.47484863  0.09872586]]
Intercept(s): [ 0.61747672 -1.67052233 -0.12705185 -2.12521444 -3.38378102]


NameError: name 'X' is not defined